In [1]:
import sys
import os
from io import StringIO
import contextlib

pwd = os.path.dirname(os.path.realpath('__file__'))
print(pwd)

sys.path.append(pwd + "../")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "geodjango.settings")
import django
import numpy as np
import pandas as pd
import pyflux as pf
import types
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import MO
django.setup()
from django.db.models import Count, Sum, Avg, Q, Func, F
from taxi.models import Trip,Edge,District

D:\DataMiningLab\geodjango


In [2]:
PATH = 'edges_fig/'
def edges():
    return Edge.objects.all().filter(tripset = 29)

class Day(Func):
    function = 'date_trunc'
    template = '%(function)s(\'day\',%(expressions)s)'
    
def colname(tail,head):
    return 'd'+str(tail)+'_d'+str(head)

def plot_fit(self, data_name = None, model='ARIMA', **kwargs):
    """ 
    Plots the fit of the model against the data
    """
    import seaborn as sns
    import matplotlib.dates as mdates
    from matplotlib.dates import MO
    months = mdates.MonthLocator()  # every month\
    weeks = mdates.WeekdayLocator(byweekday=MO)
    monthsFmt = mdates.DateFormatter('%m')

    figsize = kwargs.get('figsize',(10,7))
    plt.figure(figsize=figsize)
    date_index = self.index[max(self.ar, self.ma):self.data_length].to_pydatetime()
    mu, Y = self._model(self.latent_variables.get_z_values())

    # Catch specific family properties (imply different link functions/moments)
    if self.model_name2 == "Exponential":
        values_to_plot = 1.0/self.link(mu)
    elif self.model_name2 == "Skewt":
        t_params = self.transform_z()
        model_scale, model_shape, model_skewness = self._get_scale_and_shape(t_params)
        m1 = (np.sqrt(model_shape)*sp.gamma((model_shape-1.0)/2.0))/(np.sqrt(np.pi)*sp.gamma(model_shape/2.0))
        additional_loc = (model_skewness - (1.0/model_skewness))*model_scale*m1
        values_to_plot = mu + additional_loc
    else:
        values_to_plot = self.link(mu)

    plt.plot_date(date_index, Y,'-', label='Data')
    plt.plot_date(date_index, values_to_plot,'-', label=model+' model', c='black')
    if data_name is not None:
        plt.title(data_name)
    else:
        plt.title(self.data_name)#(label())#
    plt.legend(loc=2)
    
@contextlib.contextmanager
def stdoutIO(stdout=None):
    old = sys.stdout
    if stdout is None:
        stdout = StringIO()
    sys.stdout = stdout
    yield stdout
    sys.stdout = old
    
class EdgeTimeSeries():
    def __init__(self,tail,head,df_all_ets = None):
        self.tail = tail
        self.head = head
        self.neighbour_tail = edges().filter(tail=tail).exclude(head=tail).exclude(head=head).order_by('-weight')[:3].values_list('head',flat=True)
        self.neighbour_head = edges().filter(head=head).exclude(tail=head).exclude(tail=tail).order_by('-weight')[:3].values_list('tail',flat=True)
        if df_all_ets is not None:
            self.data = df_all_ets[self.cols()]
    def set_data(self,df_all_ets):
        self.data = df_all_ets[self.cols()]
    def cols(self):
        return list(set([colname(self.tail,h) for h in self.neighbour_tail] +  \
                        [colname(t,self.head) for t in self.neighbour_head] +  \
                        [self.target(),self.target(),self.target_inv()]))
    def label(self):
        return District.objects.get(pk=self.tail).name+'->'+District.objects.get(pk=self.head).name
    def target(self):
        return colname(self.tail,self.head)
    def target_inv(self):
        return colname(self.head,self.tail)
    def formula(self):
        return self.target()+'~1+'+'+'.join([str(x) for x in self.data.columns.values[self.data.columns.values!=self.target()]])
    def plot_serie(self):
        fig, ax = plt.subplots(figsize=(25, 10))
        months = mdates.MonthLocator()  # every month\
        weeks = mdates.WeekdayLocator(byweekday=MO)
        monthsFmt = mdates.DateFormatter('%m')
        ax.xaxis.set_major_formatter(monthsFmt)
        ax.xaxis.set_minor_locator(weeks)
        ax.set_xlabel("Month")
        ax.set_ylabel("Number of trips")
        ax.plot_date(self.data.index.to_pydatetime(), self.data.values, '-')
        ax.plot_date(self.data.index.to_pydatetime(), self.data[[self.target()]].values, '-',label=self.label())
        ax.legend(loc='upper left')
        ax.autoscale(enable=True, axis='y', tight=True)
        ax.set_ylim(bottom=0)
        plt.savefig(PATH+self.target()+".png",bbox_inches='tight')
    def plot_predict(self,mode = 'ARIMAX', h=10, past_values=10, intervals=True, **kwargs):
        figsize=(20,5)
        time = self.data.index.to_pydatetime()
        val =  self.data[self.target()].values
        time_steps = [-10,-20,-30]
        for idx, ts in enumerate(time_steps):
            if mode is 'ARIMA':
                m = pf.ARIMA(data=self.data[:ts],target = self.target(), ar=7, ma=7, family=pf.Normal())
                m.fit("MLE")
                m.plot_predict(h, past_values, intervals,figsize=figsize, **kwargs)
            else:
                m = pf.ARIMAX(data=self.data[:ts], ar=7, ma=7,formula=self.formula(),family=pf.Normal())
                m.fit("MLE")
                m.plot_predict(h, past_values, intervals,figsize=figsize,oos_data=self.data[ts:], **kwargs)
            plt.plot_date(time, val, '-',label=self.label())
            plt.xlim(pd.Timestamp('2016-05-31'), pd.Timestamp('2016-06-30'))
            plt.legend()
            plt.savefig(PATH+self.target()+'_'+mode+'_'+str(idx)+".png",bbox_inches='tight')
    def arima(self):
        self.arima = pf.ARIMA(data=self.data,target = self.target(), ar=7, ma=7, family=pf.Normal())
        self.arima.plot_fit = types.MethodType(plot_fit, self.arima)
        self.arima_res = self.arima.fit("MLE")
        with stdoutIO() as s:
            self.arima_res.summary()
        with open(PATH+self.target()+"_ARIMA.txt", "w") as text_file:
            text_file.write(s.getvalue())
        self.arima.plot_fit(figsize=(25,5),model='ARIMA')
        plt.savefig(PATH+self.target()+"_ARIMA_fit.png",bbox_inches='tight')
        self.plot_predict('ARIMA')
    def arimax(self):
        self.arimax = pf.ARIMAX(data=self.data,ar=7, ma=7,formula=self.formula(), family=pf.Normal())
        self.arimax.plot_fit = types.MethodType(plot_fit, self.arimax)
        self.arimax_res = self.arimax.fit("MLE")
        with stdoutIO() as s:
            self.arimax_res.summary()
        with open(PATH+self.target()+"_ARIMAX.txt", "w") as text_file:
            text_file.write(s.getvalue())
        self.arimax.plot_fit(figsize=(25,5),model='ARIMAX')
        plt.savefig(PATH+self.target()+"_ARIMAX_fit.png",bbox_inches='tight')
        self.plot_predict('ARIMAX')

In [ ]:
group_by_tail = edges().values_list('tail').annotate(weight=Sum('weight')).order_by('-weight')
district_selected = list(group_by_tail[:9].values_list('tail',flat=True))+list(group_by_tail.filter(tail=170)[:1].values_list('tail',flat=True))

q = None #Q(pickupDistrict=0)
for t in district_selected:
    for h in district_selected:
        ets = EdgeTimeSeries(t,h)
        if q is None:
            q = Q(Q(pickupDistrict=t),Q(dropoffDistrict__in=ets.neighbour_tail))|  \
             Q(Q(dropoffDistrict=h),Q(pickupDistrict__in=ets.neighbour_head))|  \
             Q(Q(dropoffDistrict=t),Q(pickupDistrict=h))|  \
             Q(Q(dropoffDistrict=h),Q(pickupDistrict=t))
        else:
            q |= Q(Q(pickupDistrict=t),Q(dropoffDistrict__in=ets.neighbour_tail))|  \
                 Q(Q(dropoffDistrict=h),Q(pickupDistrict__in=ets.neighbour_head))|  \
                 Q(Q(dropoffDistrict=t),Q(pickupDistrict=h))|  \
                 Q(Q(dropoffDistrict=h),Q(pickupDistrict=t))

neighbour_day = Trip.objects.all().filter(q).values('pickupDistrict','dropoffDistrict').order_by(). \
        annotate(weight=Count('pk'),total_amount=Avg('totalAmount'),day=Day('pickupTime'))
neighbour_day_array = np.array(list(neighbour_day.values_list('day','pickupDistrict','dropoffDistrict','weight'))) 

In [10]:
neighbour_day_tab = pd.pivot_table(pd.DataFrame(neighbour_day_array), values=3, index=0,
                     columns=[1,2], aggfunc=np.sum).fillna(0)
#neighbour_day_tab = pd.read_pickle('daily_table')
neighbour_day_tab.columns = [neighbour_day_tab.columns.map('d{0[0]}_d{0[1]}'.format)]
daily_table_full = pd.DataFrame(neighbour_day_tab.values)
daily_table_full.index = neighbour_day_tab.index#pd.DatetimeIndex(.values.astype(datetime),dtype=datetime)
daily_table_full.columns = neighbour_day_tab.columns.get_level_values(0)
daily_table_full.to_pickle('daily_table_full')

In [5]:
daily_table_full = pd.read_pickle('daily_table_full')

In [6]:
class GraphTimeSeries():
    def __init__(self,daily_table_full,n_hotspots=10):
        group_by_tail = edges().values_list('tail').annotate(weight=Sum('weight')).order_by('-weight')
        district_selected = list(group_by_tail[:n_hotspots-1].values_list('tail',flat=True))+list(group_by_tail.filter(tail=170)[:1].values_list('tail',flat=True))
        self.districts = district_selected
        self.edges = []
        for t in district_selected:
            for h in district_selected:
                self.edges.append(EdgeTimeSeries(t,h,daily_table_full))
    def analyze(self):
        for idx, ets in enumerate(self.edges):
            if idx > 79:
                print(idx)
                ets.plot_serie()
                ets.arima()
                ets.arimax()
                import gc
                del ets
                gc.collect()

In [8]:
taxi_mobility = GraphTimeSeries(daily_table_full)
taxi_mobility.analyze()

[190, 228, 219, 183, 100, 145, 259, 218, 151, 170]